In [17]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [18]:
%cd /content/drive/MyDrive/CV

/content/drive/.shortcut-targets-by-id/1wu7ijWhY1_HJddIenGyM7F6fvOStsSds/CV


In [19]:
import os
import pandas as pd
from PIL import Image
from tqdm import tqdm
import numpy as np
import shutil
from shutil import copyfile

In [20]:
data_path = 'Data'
os.listdir(data_path)

['nightSequence1',
 'Annotations',
 'sample-nightClip1',
 'nightSequence2',
 'sample-dayClip6',
 'daySequence1',
 'dayTrain',
 'nightTrain',
 'daySequence2']

In [21]:
night_train_path = 'Data/Annotations/Annotations/nightSequence1/'

In [22]:
df = pd.read_csv(os.path.join(night_train_path,'frameAnnotationsBOX.csv'),sep=';')

In [23]:
df

,Filename,Annotation tag,Upper left corner X,Upper left corner Y,Lower right corner X,Lower right corner Y,Origin file,Origin frame number,Origin track,Origin track frame number
0,nightTest/nightSequence1--00000.jpg,go,667,385,676,400,nightTest/nightSequence1/testSequence1.mp4,0,nightTest/nightSequence1/testSequence1.mp4,0
1,nightTest/nightSequence1--00000.jpg,stop,520,387,532,408,nightTest/nightSequence1/testSequence1.mp4,0,nightTest/nightSequence1/testSequence1.mp4,0
2,nightTest/nightSequence1--00000.jpg,stop,725,385,740,412,nightTest/nightSequence1/testSequence1.mp4,0,nightTest/nightSequence1/testSequence1.mp4,0
3,nightTest/nightSequence1--00001.jpg,go,667,385,676,400,nightTest/nightSequence1/testSequence1.mp4,1,nightTest/nightSequence1/testSequence1.mp4,1
4,nightTest/nightSequence1--00001.jpg,stop,520,386,532,407,nightTest/nightSequence1/testSequence1.mp4,1,nightTest/nightSequence1/testSequence1.mp4,1
...,...,...,...,...,...,...,...,...,...,...
18509,nightTest/nightSequence1--04860.jpg,go,873,86,1018,206,nightTest/nightSequence1/testSequence1.mp4,4860,nightTest/nightSequence1/testSequence1.mp4,4860
18510,nightTest/nightSequence1--04861.jpg,go,946,74,1091,188,nightTest/nightSequence1/testSequence1.mp4,4861,nightTest/nightSequence1/testSequence1.mp4,4861
18511,nightTest/nightSequence1--04862.jpg,go,1016,51,1161,177,nightTest/nightSequence1/testSequence1.mp4,4862,nightTest/nightSequence1/testSequence1.mp4,4862
18512,nightTest/nightSequence1--04863.jpg,go,1085,42,1245,159,nightTest/nightSequence1/testSequence1.mp4,4863,nightTest/nightSequence1/testSequence1.mp4,4863


In [24]:
df = df.drop(['Origin file','Origin track','Origin track frame number'],axis=1)

In [34]:
def changeFilename(x):
    filename = x.Filename
    
    splitted = filename.split('/')
    clipName = splitted[-1].split('--')[0]
    return os.path.join(data_path,f'nightSequence1/nightSequence1/frames/{splitted[-1]}')

df['Filename'] = df.apply(changeFilename,axis=1)
df.columns = ['image_id','label','x_min','y_min','x_max','y_max','frame']

df

,image_id,label,x_min,y_min,x_max,y_max,frame
0,Data/nightSequence1/nightSequence1/frames/nigh...,go,667,385,676,400,0
1,Data/nightSequence1/nightSequence1/frames/nigh...,stop,520,387,532,408,0
2,Data/nightSequence1/nightSequence1/frames/nigh...,stop,725,385,740,412,0
3,Data/nightSequence1/nightSequence1/frames/nigh...,go,667,385,676,400,1
4,Data/nightSequence1/nightSequence1/frames/nigh...,stop,520,386,532,407,1
...,...,...,...,...,...,...,...
18509,Data/nightSequence1/nightSequence1/frames/nigh...,go,873,86,1018,206,4860
18510,Data/nightSequence1/nightSequence1/frames/nigh...,go,946,74,1091,188,4861
18511,Data/nightSequence1/nightSequence1/frames/nigh...,go,1016,51,1161,177,4862
18512,Data/nightSequence1/nightSequence1/frames/nigh...,go,1085,42,1245,159,4863


In [35]:
label_to_idx = {'go':0, 'warning':1, 'stop': 2}
idx_to_label = {v:k for k,v in label_to_idx.items()}

def changeAnnotation(x):
    if 'go' in x['label']:
        return label_to_idx['go']
    elif 'warning' in x['label']:
        return label_to_idx['warning']
    elif 'stop' in x['label']:
        return label_to_idx['stop']
    
df['label'] = df.apply(changeAnnotation,axis=1)

annotation_tags = df['label'].unique()

In [37]:
df['clipNames'] = df[['image_id']].applymap(lambda x: x.split('/')[4])
df['clipNames'].unique()

array(['nightSequence1--00000.jpg', 'nightSequence1--00001.jpg',
       'nightSequence1--00002.jpg', ..., 'nightSequence1--04862.jpg',
       'nightSequence1--04863.jpg', 'nightSequence1--04864.jpg'],
      dtype=object)

In [38]:
df

,image_id,label,x_min,y_min,x_max,y_max,frame,clipNames
0,Data/nightSequence1/nightSequence1/frames/nigh...,0,667,385,676,400,0,nightSequence1--00000.jpg
1,Data/nightSequence1/nightSequence1/frames/nigh...,2,520,387,532,408,0,nightSequence1--00000.jpg
2,Data/nightSequence1/nightSequence1/frames/nigh...,2,725,385,740,412,0,nightSequence1--00000.jpg
3,Data/nightSequence1/nightSequence1/frames/nigh...,0,667,385,676,400,1,nightSequence1--00001.jpg
4,Data/nightSequence1/nightSequence1/frames/nigh...,2,520,386,532,407,1,nightSequence1--00001.jpg
...,...,...,...,...,...,...,...,...
18509,Data/nightSequence1/nightSequence1/frames/nigh...,0,873,86,1018,206,4860,nightSequence1--04860.jpg
18510,Data/nightSequence1/nightSequence1/frames/nigh...,0,946,74,1091,188,4861,nightSequence1--04861.jpg
18511,Data/nightSequence1/nightSequence1/frames/nigh...,0,1016,51,1161,177,4862,nightSequence1--04862.jpg
18512,Data/nightSequence1/nightSequence1/frames/nigh...,0,1085,42,1245,159,4863,nightSequence1--04863.jpg


In [39]:
df.drop(['image_id','frame'],axis=1)

,label,x_min,y_min,x_max,y_max,clipNames
0,0,667,385,676,400,nightSequence1--00000.jpg
1,2,520,387,532,408,nightSequence1--00000.jpg
2,2,725,385,740,412,nightSequence1--00000.jpg
3,0,667,385,676,400,nightSequence1--00001.jpg
4,2,520,386,532,407,nightSequence1--00001.jpg
...,...,...,...,...,...,...
18509,0,873,86,1018,206,nightSequence1--04860.jpg
18510,0,946,74,1091,188,nightSequence1--04861.jpg
18511,0,1016,51,1161,177,nightSequence1--04862.jpg
18512,0,1085,42,1245,159,nightSequence1--04863.jpg


In [40]:
def save_to_txt(dataFrame):
  for index, row in df.iterrows():
    label = row['label']
    x_min = row['x_min']
    x_max = row['x_max']
    y_min = row['y_min']
    y_max = row['y_max']
    file_name = os.path.splitext(row['clipNames'])[0]

    x_center = ((x_max + x_min) / 2) / 1280
    y_center = ((y_max + y_min) / 2) / 960
    width = (x_max - x_min) / 1280
    height = (y_max - y_min) / 960

    with open(f'{file_name}.txt', 'a') as txt_file:
        txt_file.write(f'{label} {x_center} {y_center} {width} {height}\n')

In [41]:
%cd /content/drive/MyDrive/CV/night

/content/drive/.shortcut-targets-by-id/1wu7ijWhY1_HJddIenGyM7F6fvOStsSds/CV/night


In [42]:
save_to_txt(df)

In [46]:
len(os.listdir('/content/drive/MyDrive/CV/labels'))

4759

In [47]:
image_folder = '/content/drive/MyDrive/CV/Data/nightSequence1/nightSequence1/frames' 
txt_folder = '/content/drive/MyDrive/CV/night/labels/'  

for txt_file in os.listdir(txt_folder):
    txt_file_path = os.path.join(txt_folder, txt_file)
    image_file = os.path.splitext(txt_file)[0] + '.jpg'

    image_file_path = os.path.join(image_folder, image_file)

    if os.path.isfile(image_file_path):
      
        destination_path = os.path.join('/content/drive/MyDrive/CV/night/images', image_file)
        shutil.copy(image_file_path, destination_path)

In [48]:
list_label = os.listdir('/content/drive/MyDrive/CV/night/labels')
len(list_label)

4759

In [49]:
list_img = os.listdir('/content/drive/MyDrive/CV/night/images')
len(list_img)

4759

In [50]:
import random

def split_txt_folder(input_folder, output_train, output_test, output_valid, split_ratio):
    for file_name in os.listdir(input_folder):
        file_path = os.path.join(input_folder, file_name)

        choice = random.choices(['train', 'test', 'valid'], split_ratio)[0]
        if choice == 'train':
            destination_folder = output_train
        elif choice == 'test':
            destination_folder = output_test
        else:
            destination_folder = output_valid

        destination_path = os.path.join(destination_folder, file_name)
        shutil.copyfile(file_path, destination_path)

folder = '/content/drive/MyDrive/CV/night/images'

train = '/content/drive/MyDrive/CV/Data_YOLO/train/images/'
test = '/content/drive/MyDrive/CV/Data_YOLO/test/images/'
valid = '/content/drive/MyDrive/CV/Data_YOLO/valid/images/'

split_ratio = [0.7, 0.1, 0.2]

split_txt_folder(folder, train, test, valid, split_ratio)

In [51]:
list_images = os.listdir('/content/drive/MyDrive/CV/Data_YOLO/train/images')
len(list_images)

9040

In [52]:
list_images = os.listdir('/content/drive/MyDrive/CV/Data_YOLO/test/images')
len(list_images)

1305

In [53]:
list_images = os.listdir('/content/drive/MyDrive/CV/Data_YOLO/valid/images')
len(list_images)

2605

In [54]:
def img_to_label(folder):
  image_folder = f'/content/drive/MyDrive/CV/Data_YOLO/{folder}/images/' 
  txt_folder = '/content/drive/MyDrive/CV/night/labels/'  

  for image_file in os.listdir(image_folder):
      image_file_path = os.path.join(image_folder, image_file)
      txt_file = os.path.splitext(image_file)[0] + '.txt'

      txt_file_path = os.path.join(txt_folder, txt_file)

      if os.path.isfile(txt_file_path):
        
          destination_path = os.path.join(f'/content/drive/MyDrive/CV/Data_YOLO/{folder}/labels/', txt_file)
          shutil.copy(txt_file_path, destination_path)

In [55]:
folder = ['train','test','valid']
for i in folder:
  img_to_label(i)

In [56]:
list_label = os.listdir('/content/drive/MyDrive/CV/Data_YOLO/train/labels/')
len(list_label)

9040

In [57]:
list_label = os.listdir('/content/drive/MyDrive/CV/Data_YOLO/test/labels/')
len(list_label)

1305

In [58]:
list_label = os.listdir('/content/drive/MyDrive/CV/Data_YOLO/valid/labels/')
len(list_label)

2605